<a href="https://colab.research.google.com/github/serdarbozoglan/My_Pyspark/blob/master/SparkNLP_Training03_NamedEntityRecognition(NER)_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code is from GitHub

https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/3.SparkNLP_Pretrained_Models.ipynb

Eger daha once indirdigimiz local'de bulunan bir pretrainedModel varsa onu cagirirken **.pretrained()** yerine **.load()** kullanabiliriz

In [0]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.4.5

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)
     |████████████████████████████████| 215.7MB 69kB/s 
     |████████████████████████████████| 204kB 52.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=b681aa5ffb36eecebcdae5b5cbb311ca99ca4ab3c6bc2e8e201b4dd47372cdfa
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 112kB 9.2MB/s 


In [0]:
import pandas as pd

## Start Spark

In [0]:
import sparknlp
#secret = '2NPvFO6B2a'
#spark = start(secret)
spark = sparknlp.start(gpu=True)

from sparknlp.base import *
from sparknlp.annotator import *

print('Spark NLP Version : ', sparknlp.version())
print('Apache Spark Version :', spark.version)

Spark NLP Version :  2.4.5
Apache Spark Version : 2.4.4


## CoNLL Data Prep

In [0]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.train
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.testa

--2020-04-21 03:21:30--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘eng.train’

eng.train           100%[===================>]   3.13M  --.-KB/s    in 0.04s   

2020-04-21 03:21:31 (74.3 MB/s) - ‘eng.train’ saved [3283420/3283420]

--2020-04-21 03:21:34--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 O

In [0]:
with open("eng.train") as f:
    train_txt =f.read()

print (train_txt[:500])
## CoNLL Data Format always starts with "-DOCSTART- -X- -X- O" and should be in the format below

-DOCSTART- -X- -X- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O
. . O O

Peter NNP B-NP B-PER
Blackburn NNP I-NP I-PER

BRUSSELS NNP B-NP B-LOC
1996-08-22 CD I-NP O

The DT B-NP O
European NNP I-NP B-ORG
Commission NNP I-NP I-ORG
said VBD B-VP O
on IN B-PP O
Thursday NNP B-NP O
it PRP B-NP O
disagreed VBD B-VP O
with IN B-PP O
German JJ B-NP B-MISC
advice NN I-NP O
to TO B-PP O
consumers NNS B-NP


In [0]:
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng.train')
training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|EU rejects German...|[[document, 0, 47...|[[document, 0, 47...|[[token, 0, 1, EU...|[[pos, 0, 1, NNP,...|[[named_entity, 0...|
|     Peter Blackburn|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 4, Pe...|[[pos, 0, 4, NNP,...|[[named_entity, 0...|
| BRUSSELS 1996-08-22|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 7, BR...|[[pos, 0, 7, NNP,...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [0]:
training_data.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = tr

In [0]:
training_data.count()
# 14K sentences

14041

In [0]:
import pyspark.sql.functions as F

training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("pos"),
        F.expr("cols['2']").alias("ner_label")).show(truncate=50)

+----------+---+---------+
|     token|pos|ner_label|
+----------+---+---------+
|        EU|NNP|    B-ORG|
|   rejects|VBZ|        O|
|    German| JJ|   B-MISC|
|      call| NN|        O|
|        to| TO|        O|
|   boycott| VB|        O|
|   British| JJ|   B-MISC|
|      lamb| NN|        O|
|         .|  .|        O|
|     Peter|NNP|    B-PER|
| Blackburn|NNP|    I-PER|
|  BRUSSELS|NNP|    B-LOC|
|1996-08-22| CD|        O|
|       The| DT|        O|
|  European|NNP|    B-ORG|
|Commission|NNP|    I-ORG|
|      said|VBD|        O|
|        on| IN|        O|
|  Thursday|NNP|        O|
|        it|PRP|        O|
+----------+---+---------+
only showing top 20 rows



In [0]:
training_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)

+------------+------+
|ground_truth|count |
+------------+------+
|O           |169578|
|B-LOC       |7140  |
|B-PER       |6600  |
|B-ORG       |6321  |
|I-PER       |4528  |
|I-ORG       |3704  |
|B-MISC      |3438  |
|I-LOC       |1157  |
|I-MISC      |1155  |
+------------+------+



In [0]:
# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)

glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [0]:
# ner_label will be our target column
# .setPo = Dropout ratio
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(6)\
  .setLr(0.001)\
  .setPo(0.005)\
  .setBatchSize(32)\
  .setRandomSeed(0)\
  .setVerbose(1)\
  .setValidationSplit(0.2)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setIncludeConfidence(True)

ner_pipeline = Pipeline(stages=[
          glove_embeddings,
          nerTagger
 ])

## Fitting

In [0]:
%%time

ner_model = ner_pipeline.fit(training_data)
# if you get an error for incompatible TF graph, use 4.1 NerDL-Graph.ipynb notebook to create a graph 
# from GitHub repository

CPU times: user 104 ms, sys: 27.5 ms, total: 132 ms
Wall time: 9min 31s


In [0]:
!cd ~/annotator_logs && ls -lt

total 4
-rw-r--r-- 1 root root 876 Apr 21 03:36 NerDLApproach_b84a0e7cb18b.log


In [0]:
!cat ~/annotator_logs/NerDLApproach_cc78216e548c.log

Name of the selected graph: ner-dl/blstm_10_100_128_120.pb
Training started, trainExamples: 11233, labels: 9 chars: 84, 


Epoch: 0 started, learning rate: 0.001, dataset size: 11233
Done, 83.310741104 loss: 338.20844, batches: 26
Quality on validation dataset (20.0%), valExamples = 2808
time to finish evaluation: 8.802185197
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 323	 113	 1107	 0.7408257	 0.22587413	 0.34619507
I-ORG	 5	 8	 779	 0.3846154	 0.0063775512	 0.0125470515
I-MISC	 0	 0	 225	 0.0	 0.0	 0.0
I-LOC	 0	 0	 247	 0.0	 0.0	 0.0
I-PER	 229	 36	 657	 0.86415094	 0.25846502	 0.3979149
B-MISC	 0	 0	 670	 0.0	 0.0	 0.0
B-ORG	 333	 237	 947	 0.5842105	 0.26015624	 0.35999998
B-PER	 289	 52	 1019	 0.84750736	 0.22094801	 0.35051548
tp: 1179 fp: 446 fn: 5651 labels: 8
Macro-average	 prec: 0.42766374, rec: 0.12147762, f1: 0.1892102
Micro-average	 prec: 0.72553843, rec: 0.17262079, f1: 0.27888826


Epoch: 1 started, learning rate: 9.950249E-4, dataset size: 11233
Done, 86.65745414 loss: 199

In [0]:
from sparknlp.training import CoNLL

test_data = CoNLL().readDataset(spark, './eng.testa')

test_data = glove_embeddings.transform(test_data)

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|          embeddings|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|CRICKET - LEICEST...|[[document, 0, 64...|[[document, 0, 64...|[[token, 0, 6, CR...|[[pos, 0, 6, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|   LONDON 1996-08-30|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 5, LO...|[[pos, 0, 5, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|West Indian all-r...|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 3, We...|[[pos, 0, 3, NNP,...|[[named_entity, 0...|[[word_embeddings...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
# transform equals predict in sklearn
predictions = ner_model.transform(test_data)
predictions.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|          embeddings|                 ner|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|CRICKET - LEICEST...|[[document, 0, 64...|[[document, 0, 64...|[[token, 0, 6, CR...|[[pos, 0, 6, NNP,...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|   LONDON 1996-08-30|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 5, LO...|[[pos, 0, 5, NNP,...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|West Indian all-r...|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 3, We...|[[pos, 0, 3, NNP,...|[[named_entity, 0...|[[word_embeddings...|[[

In [0]:
predictions.select('token.result', 'label.result', 'ner.result').show(3, truncate=50)

+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|                                            result|                                            result|                                            result|
+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|[CRICKET, -, LEICESTERSHIRE, TAKE, OVER, AT, TO...|             [O, O, B-ORG, O, O, O, O, O, O, O, O]|         [B-LOC, O, B-ORG, O, O, O, O, O, O, O, O]|
|                              [LONDON, 1996-08-30]|                                        [B-LOC, O]|                                        [B-LOC, O]|
|[West, Indian, all-rounder, Phil, Simmons, took...|[B-MISC, I-MISC, O, B-PER, I-PER, O, O, O, O, O...|[B-LOC, O, O, B-PER, I-PER, O, O, O, O, O, O, O...|
+--------------------------------------------------+------------------

## Test Evaluation

In [0]:
import pyspark.sql.functions as F

predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth"),
        F.expr("cols['2']").alias("prediction")).show(truncate=False)

+--------------+------------+----------+
|token         |ground_truth|prediction|
+--------------+------------+----------+
|CRICKET       |O           |B-LOC     |
|-             |O           |O         |
|LEICESTERSHIRE|B-ORG       |B-ORG     |
|TAKE          |O           |O         |
|OVER          |O           |O         |
|AT            |O           |O         |
|TOP           |O           |O         |
|AFTER         |O           |O         |
|INNINGS       |O           |O         |
|VICTORY       |O           |O         |
|.             |O           |O         |
|LONDON        |B-LOC       |B-LOC     |
|1996-08-30    |O           |O         |
|West          |B-MISC      |B-LOC     |
|Indian        |I-MISC      |O         |
|all-rounder   |O           |O         |
|Phil          |B-PER       |B-PER     |
|Simmons       |I-PER       |I-PER     |
|took          |O           |O         |
|four          |O           |O         |
+--------------+------------+----------+
only showing top

## Below Codes Belong To Previous Notebooks

## LemmatizerModel

In [0]:

!wget -O news_category_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv

--2020-04-21 03:01:42--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.27.14
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.27.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1504408 (1.4M) [text/csv]
Saving to: ‘news_category_test.csv’

news_category_test. 100%[===================>]   1.43M  1.32MB/s    in 1.1s    

2020-04-21 03:01:44 (1.32 MB/s) - ‘news_category_test.csv’ saved [1504408/1504408]



In [0]:
import pyspark.sql.functions as F

news_df = spark.read \
        .option("header", 'true') \
        .csv("news_category_test.csv") \
        .withColumnRenamed("description", 'text')

news_df.show(truncate=50)    

+--------+--------------------------------------------------+
|category|                                              text|
+--------+--------------------------------------------------+
|Business|Unions representing workers at Turner   Newall ...|
|Sci/Tech| TORONTO, Canada    A second team of rocketeers...|
|Sci/Tech| A company founded by a chemistry researcher at...|
|Sci/Tech| It's barely dawn when Mike Fitzpatrick starts ...|
|Sci/Tech| Southern California's smog fighting agency wen...|
|Sci/Tech|"The British Department for Education and Skill...|
|Sci/Tech|"confessed author of the Netsky and Sasser viru...|
|Sci/Tech|\\FOAF/LOAF  and bloom filters have a lot of in...|
|Sci/Tech|"Wiltshire Police warns about ""phishing"" afte...|
|Sci/Tech|In its first two years, the UK's dedicated card...|
|Sci/Tech| A group of technology companies  including Tex...|
|Sci/Tech| Apple Computer Inc.&lt;AAPL.O&gt; on  Tuesday ...|
|Sci/Tech| Free Record Shop, a Dutch music  retail chain,...|
|Sci/Tec

In [0]:
 # lemma_antbnc is coming from JohnsonLab Github page there is the list of pretrained Models

lemmatizer = LemmatizerModel.pretrained('lemma_antbnc', 'en')\
            .setInputCols(["token"])\
            .setOutputCol("lemma")

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [0]:
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
        .setInputCol('text')\
        .setOutputCol('document')

tokenizer = Tokenizer()\
        .setInputCols(['document'])\
        .setOutputCol('token')

stemmer = Stemmer()\
        .setInputCols(['token'])\
        .setOutputCol('stem')

nlpPipeline = Pipeline(stages=[
              documentAssembler,
              tokenizer,
              stemmer,
              lemmatizer          
])

empty_df = spark.createDataFrame([['']]).toDF('text')
pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df)
result.show(5, truncate=100)

+--------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|category|                                                                                                text|                                                                                            document|                                                                                               token|                                                                                                stem|                                                                      

## PerceptronModel(POS- Part of Speech Tags)

In [0]:
pos = PerceptronModel.pretrained('pos_anc', 'en')\
        .setInputCols(['document', 'token'])\
        .setOutputCol('pos')

pos_anc download started this may take some time.
Approximate size to download 4.3 MB
[OK!]


In [0]:
nlpPipeline = Pipeline(stages=[
              documentAssembler,
              tokenizer,
              stemmer,
              lemmatizer,
              pos         
])

empty_df = spark.createDataFrame([['']]).toDF('text')
pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df)
result.show(5, truncate=100)

+--------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|category|                                                                                                text|                                                                                            document|                                                                                               token|                                                                      

In [0]:
result.select('token.result', 'pos.result').show(10, truncate=60)

+------------------------------------------------------------+------------------------------------------------------------+
|                                                      result|                                                      result|
+------------------------------------------------------------+------------------------------------------------------------+
|[Unions, representing, workers, at, Turner, Newall, say, ...|[NNP, VBG, NNS, IN, NNP, NNP, VBP, PRP, VBP, POS, JJ, POS...|
|[TORONTO, ,, Canada, A, second, team, of, rocketeers, com...|[NNP, ,, NNP, DT, JJ, NN, IN, NNS, VBG, IN, DT, NN, CD, N...|
|[A, company, founded, by, a, chemistry, researcher, at, t...|[DT, NN, VBN, IN, DT, NN, NN, IN, DT, NNP, IN, NNP, VBD, ...|
|[It's, barely, dawn, when, Mike, Fitzpatrick, starts, his...|[NNP, RB, NN, WRB, NNP, NNP, VBZ, PRP$, NN, IN, DT, NN, I...|
|[Southern, California's, smog, fighting, agency, went, af...|[NNP, NNP, NN, VBG, NN, VBD, IN, NNS, IN, DT, NN, NN, NNP...|
|[", The

In [0]:
result = pipelineModel.transform(news_df.limit(100))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'token.begin', 'token.end', 'stem.result',  'lemma.result', 'pos.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']").alias("stem"),
        F.expr("cols['4']").alias("lemma"),
        F.expr("cols['5']").alias("pos")).toPandas()

result_df.head(10)

,token,begin,end,stem,lemma,pos
0,Unions,0,5,union,Unions,NNP
1,representing,7,18,repres,represent,VBG
2,workers,20,26,worker,worker,NNS
3,at,28,29,at,at,IN
4,Turner,31,36,turner,Turner,NNP
5,Newall,40,45,newal,Newall,NNP
6,say,47,49,sai,say,VBP
7,they,51,54,thei,they,PRP
8,are,56,58,ar,be,VBP
9,',60,60,',',POS


In [0]:
# same in LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate('Unions representing workers at Turner Newall say there are lots of books there.')

In [0]:
list(zip(light_result['token'], light_result['stem'], light_result['lemma'], light_result['pos']))

[('Unions', 'union', 'Unions', 'NNP'),
 ('representing', 'repres', 'represent', 'VBG'),
 ('workers', 'worker', 'worker', 'NNS'),
 ('at', 'at', 'at', 'IN'),
 ('Turner', 'turner', 'Turner', 'NNP'),
 ('Newall', 'newal', 'Newall', 'NNP'),
 ('say', 'sai', 'say', 'VB'),
 ('there', 'there', 'there', 'EX'),
 ('are', 'ar', 'be', 'VBP'),
 ('lots', 'lot', 'lot', 'NNS'),
 ('of', 'of', 'of', 'IN'),
 ('books', 'book', 'book', 'NNS'),
 ('there', 'there', 'there', 'RB'),
 ('.', '.', '.', '.')]

In [0]:
# applying POS chunker to find a custom pattern

chunker = Chunker()\
    .setInputCols(["document", "pos"])\
    .setOutputCol("chunk")\
    .setRegexParsers(["<NNP>+", "<DT>?<JJ>*<NN>"])

# NNP: Proper Noun
# NN: Common Noun
# DT: Determinator (e.g. the)
# JJ: Adjective

nlpPipeline = Pipeline(stages=[
              documentAssembler,
              tokenizer,
              stemmer,
              lemmatizer,
              pos,
              chunker        
])

empty_df = spark.createDataFrame([['']]).toDF('text')
pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(100))
result.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|category|                text|            document|               token|                stem|               lemma|                 pos|               chunk|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Business|Unions representi...|[[document, 0, 12...|[[token, 0, 5, Un...|[[token, 0, 5, un...|[[token, 0, 5, Un...|[[pos, 0, 5, NNP,...|[[chunk, 0, 5, Un...|
|Sci/Tech| TORONTO, Canada ...|[[document, 0, 22...|[[token, 1, 7, TO...|[[token, 1, 7, to...|[[token, 1, 7, TO...|[[pos, 1, 7, NNP,...|[[chunk, 1, 7, TO...|
|Sci/Tech| A company founde...|[[document, 0, 20...|[[token, 1, 1, A,...|[[token, 1, 1, a,...|[[token, 1, 1, A,...|[[pos, 1, 1, DT, ...|[[chunk, 52, 61, ...|
|Sci/Tech| It's barely dawn...|[[document, 0, 26...|

In [0]:
result.select('chunk').show(10,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
result_df = result.select(F.explode(F.arrays_zip('chunk.result', 'chunk.begin',  'chunk.end')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end")).toPandas()

result_df.head(10)

,chunk,begin,end
0,Unions,0,5
1,Turner Newall,31,45
2,Federal Mogul,113,125
3,stricken,92,99
4,parent,101,106
5,firm,108,111
6,TORONTO,1,7
7,Canada,10,15
8,Ansari X Prize,82,95
9,A second team,20,32


## Dependency Parser

In [0]:
dep_parser = DependencyParserModel.pretrained('dependency_conllu')\
            .setInputCols(['document', 'pos', 'token'])\
            .setOutputCol('dependency')

dependency_conllu download started this may take some time.
Approximate size to download 16.6 MB
[OK!]


In [0]:
typed_dep_parser = TypedDependencyParserModel.pretrained('dependency_typed_conllu')\
        .setInputCols(["token", "pos", "dependency"])\
        .setOutputCol("dependency_type")

dependency_typed_conllu download started this may take some time.
Approximate size to download 257.4 KB
[OK!]


In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer,
 pos,
 dep_parser,
 typed_dep_parser
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:

result = pipelineModel.transform(news_df.limit(100))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'token.begin',  'token.end', 'dependency.result', 'dependency_type.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']").alias("dependency"),
        F.expr("cols['4']").alias("dependency_type")).toPandas()

result_df.head(10)

,chunk,begin,end,dependency,dependency_type
0,Unions,0,5,ROOT,root
1,representing,7,18,workers,amod
2,workers,20,26,Unions,flat
3,at,28,29,Turner,case
4,Turner,31,36,workers,flat
5,Newall,40,45,say,nsubj
6,say,47,49,Unions,parataxis
7,they,51,54,disappointed,nsubj
8,are,56,58,disappointed,nsubj
9,',60,60,disappointed,case


## SpellChecker

In [0]:
spell_checker = NorvigSweetingModel.pretrained('spellcheck_norvig')\
        .setInputCols("token")\
        .setOutputCol("corrected")

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[OK!]


In [0]:
# spell_checker = SymmetricDeleteModel.pretrained('spellcheck_sd')\
#         .setInputCols(['token'])\
#         .setOutputCol("corrected")

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer,
 pos,
 spell_checker
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(100))

In [0]:
result_df = result.select(F.explode(F.arrays_zip('token.result', 'corrected.result', 'stem.result',  'lemma.result', 'pos.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("corrected"),
        F.expr("cols['2']").alias("stem"),
        F.expr("cols['3']").alias("lemma"),
        F.expr("cols['4']").alias("pos")).toPandas()

result_df.head(10)

,token,corrected,stem,lemma,pos
0,Unions,Unions,union,Unions,NNP
1,representing,representing,repres,represent,VBG
2,workers,workers,worker,worker,NNS
3,at,at,at,at,IN
4,Turner,Turner,turner,Turner,NNP
5,Newall,Newell,newal,Newall,NNP
6,say,say,sai,say,VBP
7,they,they,thei,they,PRP
8,are,are,ar,be,VBP
9,',',',',POS


In [0]:
light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate('The patint has pain and headace')

list(zip(light_result['token'], light_result['corrected']))

[('The', 'The'),
 ('patint', 'patient'),
 ('has', 'has'),
 ('pain', 'pain'),
 ('and', 'and'),
 ('headace', 'headache')]

In [0]:
a

NER_cb957d2304b3

## Word Embeddings (GloVe)

In [0]:
glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
                    .setInputCols(['document', 'token'])\
                    .setOutputCol('embeddings')

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [0]:
nlpPipeline = Pipeline(stages=[
                               documentAssembler,
                               tokenizer,
                               glove_embeddings
])

empty_df = spark.createDataFrame([['']]).toDF('text')
pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(100))

In [0]:
result.toPandas()

,category,text,document,token,embeddings
0,Business,Unions representing workers at Turner Newall...,"[(document, 0, 126, Unions representing worker...","[(token, 0, 5, Unions, {'sentence': '0'}, []),...","[(word_embeddings, 0, 5, Unions, {'sentence': ..."
1,Sci/Tech,"TORONTO, Canada A second team of rocketeer...","[(document, 0, 222, TORONTO, Canada A seco...","[(token, 1, 7, TORONTO, {'sentence': '0'}, [])...","[(word_embeddings, 1, 7, TORONTO, {'sentence':..."
2,Sci/Tech,A company founded by a chemistry researcher a...,"[(document, 0, 209, A company founded by a ch...","[(token, 1, 1, A, {'sentence': '0'}, []), (tok...","[(word_embeddings, 1, 1, A, {'sentence': '0', ..."
3,Sci/Tech,It's barely dawn when Mike Fitzpatrick starts...,"[(document, 0, 267, It's barely dawn when Mik...","[(token, 1, 4, It's, {'sentence': '0'}, []), (...","[(word_embeddings, 1, 4, It's, {'sentence': '0..."
4,Sci/Tech,Southern California's smog fighting agency we...,"[(document, 0, 174, Southern California's smo...","[(token, 1, 8, Southern, {'sentence': '0'}, []...","[(word_embeddings, 1, 8, Southern, {'sentence'..."
...,...,...,...,...,...
95,Sports,England coach Sven Goran Eriksson said Tuesda...,"[(document, 0, 171, England coach Sven Goran ...","[(token, 1, 7, England, {'sentence': '0'}, [])...","[(word_embeddings, 1, 7, England, {'sentence':..."
96,Sports,Striker Emile Heskey has pulled out of the En...,"[(document, 0, 176, Striker Emile Heskey has ...","[(token, 1, 7, Striker, {'sentence': '0'}, [])...","[(word_embeddings, 1, 7, Striker, {'sentence':..."
97,Business,""" Staples Inc. &lt;A HREF=""""http://www.investo...","[(document, 0, 144, "" Staples Inc. &lt;A HREF=...","[(token, 0, 0, "", {'sentence': '0'}, []), (tok...","[(word_embeddings, 0, 0, "", {'sentence': '0', ..."
98,World,"AGHDAD, Iraq, Aug. 17 A delegation of Iraqis ...","[(document, 0, 229, AGHDAD, Iraq, Aug. 17 A d...","[(token, 0, 5, AGHDAD, {'sentence': '0'}, []),...","[(word_embeddings, 0, 5, AGHDAD, {'sentence': ..."


In [0]:
result.select('token.result').take(1)

[Row(result=['Unions', 'representing', 'workers', 'at', 'Turner', 'Newall', 'say', 'they', 'are', "'", 'disappointed', "'", 'after', 'talks', 'with', 'stricken', 'parent', 'firm', 'Federal', 'Mogul', '.'])]

In [0]:
result.select('embeddings.embeddings').take(1)

[Row(embeddings=[[0.7186499834060669, 0.8075399994850159, -1.1786999702453613, 0.271450012922287, -0.48833000659942627, -0.1893800050020218, -1.1789000034332275, 0.1783600002527237, -0.21995000541210175, -0.7215999960899353, -0.5354200005531311, -0.7507500052452087, 0.41176000237464905, 0.18951000273227692, -0.5093500018119812, -0.735010027885437, 0.33823999762535095, 0.3451400101184845, -0.16735999286174774, 0.130390003323555, 1.1055999994277954, 0.5508400201797485, -0.01406800001859665, -0.5322800278663635, -0.23598000407218933, -1.0496000051498413, -0.6549199819564819, -0.17997999489307404, 0.10341999679803848, -0.315530002117157, 0.31481000781059265, 0.8438799977302551, 0.305400013923645, -1.158400058746338, 0.047777000814676285, -0.04945499822497368, 0.7908999919891357, -0.22619999945163727, -0.6294800043106079, 0.3116999864578247, -0.7399600148200989, 0.1283400058746338, 0.10723999887704849, 0.3257099986076355, -0.2687300145626068, -0.70169997215271, 0.16669000685214996, 0.330889

In [0]:
result_df = result.select(F.explode(F.arrays_zip('token.result', 'embeddings.embeddings')).alias('cols'))\
.select(F.expr("cols['0']").alias('token'),
        F.expr("cols['1']").alias('embeddings'))

result_df.show(10, truncate=100)

+------------+----------------------------------------------------------------------------------------------------+
|       token|                                                                                          embeddings|
+------------+----------------------------------------------------------------------------------------------------+
|      Unions|[0.71865, 0.80754, -1.1787, 0.27145, -0.48833, -0.18938, -1.1789, 0.17836, -0.21995, -0.7216, -0....|
|representing|[0.25671, 0.30035, -0.18006, 0.46666, 0.98501, 0.2321, -0.34959, 0.26997, -0.99667, -0.43404, -0....|
|     workers|[0.50592, 0.71717, -0.67236, -0.32112, -0.58285, -0.47977, -0.50243, 0.60594, 0.25709, 0.03974, -...|
|          at|[0.1766, 0.093851, 0.24351, 0.44313, -0.39037, 0.12524, -0.19918, 0.59855, -0.82035, 0.28006, 0.5...|
|      Turner|[0.51634, -0.37186, -0.21776, -1.0115, 0.4014, -0.4841, 0.36274, -0.2952, -0.42258, -0.62844, 0.6...|
|      Newall|[-0.38857, -1.1449, -0.41737, -0.31969, -0.16546, -0.7044,

## ELMO Embeddings

In [0]:
elmo_embeddings = ElmoEmbeddings.pretrained('elmo')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]


In [0]:
nlpPipeline = Pipeline(stages=[
                               documentAssembler,
                               tokenizer,
                               elmo_embeddings
])

empty_df = spark.createDataFrame([['']]).toDF('text')
pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(100))

In [0]:
result_df = result.select(F.explode(F.arrays_zip('token.result', 'embeddings.embeddings')).alias('cols'))\
.select(F.expr("cols['0']").alias('token'),
        F.expr("cols['1']").alias('embeddings'))

result_df.show(10, truncate=100)

+------------+----------------------------------------------------------------------------------------------------+
|       token|                                                                                          embeddings|
+------------+----------------------------------------------------------------------------------------------------+
|      Unions|[0.5389476, 0.4226915, -0.16224553, -0.1128005, -0.12524837, -0.28185326, 0.12341033, -0.22033125...|
|representing|[0.1591705, 0.95000935, -0.12492801, 0.9341251, 0.5854504, 0.7605221, -0.30143818, -0.3193327, -0...|
|     workers|[0.082813635, -0.17253807, -0.73114246, 0.56554866, 0.36294985, -0.42002633, 0.66982335, -0.71128...|
|          at|[-0.11999596, 0.01299414, -0.078183725, 0.037628517, -0.268691, 0.13629477, -0.27565888, 0.343611...|
|      Turner|[0.32819325, -0.11992816, 0.15701114, 0.560922, -0.2836563, -0.1923972, 0.04214704, 0.5058229, -0...|
|      Newall|[-0.17683865, -0.5143349, -0.45629728, -7.754117E-4, -0.35

## BERT Embedding

In [0]:
bert_embeddings = BertEmbeddings.pretrained('bert_base_uncased')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

bert_base_uncased download started this may take some time.
Approximate size to download 392.5 MB
[OK!]


In [0]:
nlpPipeline = Pipeline(stages=[
                               documentAssembler,
                               tokenizer,
                               bert_embeddings
])

empty_df = spark.createDataFrame([['']]).toDF('text')
pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(100))

In [0]:
result_df = result.select(F.explode(F.arrays_zip('token.result', 'embeddings.embeddings')).alias('cols'))\
.select(F.expr("cols['0']").alias('token'),
        F.expr("cols['1']").alias('embeddings'))

result_df.show(10, truncate=100)

+------------+----------------------------------------------------------------------------------------------------+
|       token|                                                                                          embeddings|
+------------+----------------------------------------------------------------------------------------------------+
|      Unions|[0.3423483, -0.7621089, -0.95610934, -0.9080744, 0.03278944, -0.19068101, 0.06841645, 0.8715485, ...|
|representing|[-0.6609322, 0.36830586, -0.04385671, -0.062400907, -0.21250877, -0.3625311, -0.13388896, -9.9757...|
|     workers|[-0.49138808, -0.0533655, -1.1493174, -0.6050249, -0.7109844, 0.20254612, 0.53452104, -0.6757567,...|
|          at|[-0.59642524, 0.13935125, 0.09018701, -0.6268516, -0.75282824, 0.16119936, 0.10998525, 0.6566241,...|
|      Turner|[0.87067986, 0.6809796, 0.7334639, 0.03486839, 0.07190508, 0.5168021, 0.18955632, 0.76651955, 0.5...|
|      Newall|[0.17950639, -0.34025913, 0.25570604, 0.21317199, -0.33208

## Universal Sentence Encoder

In [0]:
# We will not send "token" but only document

# no need for token columns 
use_embeddings = UniversalSentenceEncoder.pretrained('tfhub_use').\
  setInputCols(["document"]).\
  setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [0]:
nlpPipeline = Pipeline(stages=[
                               documentAssembler,
                               tokenizer,
                               use_embeddings
])

empty_df = spark.createDataFrame([['']]).toDF('text')
pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(100))

In [0]:
result_df = result.select(F.explode(F.arrays_zip('document.result', 'sentence_embeddings.embeddings')).alias('cols'))\
.select(F.expr("cols['0']").alias('document'),
        F.expr("cols['1']").alias('embeddings'))

result_df.show(10, truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                            document|                                                                                          embeddings|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stric...|[0.01299754, 0.019844763, -0.02462647, 0.039759073, -0.04424647, 0.01319757, 0.0786744, -0.053629...|
| TORONTO, Canada    A second team of rocketeers competing for the  #36;10 million Ansari X Prize,...|[0.0019999044, 0.051844064, -0.0440291, -5.932768E-4, -0.038505998, -0.02727945, 0

## Loading Models from local

In [0]:
!cd ~/cache_pretrained && pwd

/root/cache_pretrained


In [0]:
!cd ~/cache_pretrained && ls -l

total 44
drwxr-xr-x 4 root root 4096 Apr 20 20:46 bert_base_uncased_en_2.4.0_2.4_1580579889322
drwxr-xr-x 4 root root 4096 Apr 20 18:29 dependency_conllu_en_2.0.8_2.4_1561435004077
drwxr-xr-x 4 root root 4096 Apr 20 18:29 dependency_typed_conllu_en_2.0.8_2.4_1561473259215
drwxr-xr-x 3 root root 4096 Apr 20 20:14 elmo_en_2.4.0_2.4_1580488815299
drwxr-xr-x 4 root root 4096 Apr 20 19:46 glove_100d_en_2.4.0_2.4_1579690104032
drwxr-xr-x 4 root root 4096 Apr 20 17:57 lemma_antbnc_en_2.0.2_2.4_1556480454569
drwxr-xr-x 4 root root 4096 Apr 20 17:59 ner_crf_en_2.4.0_2.4_1580237286004
drwxr-xr-x 4 root root 4096 Apr 20 18:09 pos_anc_en_2.0.2_2.4_1556659930154
drwxr-xr-x 4 root root 4096 Apr 20 18:35 spellcheck_norvig_en_2.1.0_2.4_1563017660080
drwxr-xr-x 4 root root 4096 Apr 20 18:44 spellcheck_sd_en_2.1.0_2.4_1563019290368
drwxr-xr-x 3 root root 4096 Apr 20 20:50 tfhub_use_en_2.4.0_2.4_1587136330099


In [0]:
glove_embeddings = WordEmbeddings.load('/root/cache_pretrained/glove_100d_en_2.4.0_2.4_1579690104032')\
                    .setInputCols(['document' 'token'])\
                    .setOutputCol('glove_embeddings')

## Using Your Custom Embeddings in Spark NLP

In [0]:
custom_embeddings = WordEmbeddings()\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("my_embeddings")\
  .setStoragePath('PubMed-shuffle-win-2.bin', "BINARY")\
  .setDimension(200)

## Getting Sentence Embeddings from GloVe, Elmo and BERT

In [0]:
glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
                    .setInputCols(['document', 'token'])\
                    .setOutputCol('embeddings')

embeddingsSentence = SentenceEmbeddings()\
                    .setInputCols(['document', 'embeddings'])\
                    .setOutputCol('sentence_embeddings')\
                    .setPoolingStrategy("AVERAGE") # or SUM

nlpPipeline = Pipeline(stages=[
                               documentAssembler,
                               tokenizer,
                               glove_embeddings,
                               embeddingsSentence # we get average vector of vector representatios of tokens
])

empty_df = spark.createDataFrame([['']]).toDF('text')
nlpPipelineModel = nlpPipeline.fit(empty_df)

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [0]:
result = pipelineModel.transform(news_df.limit(100))

result_df = result.select(F.explode(F.arrays_zip('document.result', 'sentence_embeddings.embeddings')).alias('cols'))\
.select(F.expr("cols['0']").alias('document'),
        F.expr("cols['1']").alias('sentence_embeddings'))

result_df.show(10, truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                            document|                                                                                 sentence_embeddings|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stric...|[0.01299754, 0.019844763, -0.02462647, 0.039759073, -0.04424647, 0.01319757, 0.0786744, -0.053629...|
| TORONTO, Canada    A second team of rocketeers competing for the  #36;10 million Ansari X Prize,...|[0.0019999044, 0.051844064, -0.0440291, -5.932768E-4, -0.038505998, -0.02727945, 0

## Cosine Similarity Between 2 embeddings (Sentence Similarity)

In [0]:
from scipy.spatial import distance
import numpy as np

v1 = result_df.select('sentence_embeddings').take(2)[0][0]
v2 = result_df.select('sentence_embeddings').take(2)[0][0] # ayni cumlenin similarity'sine bkacagiz

In [0]:
# similarity = 1 - distance(cosine)
1 - distance.cosine(np.array(v1), np.array(v2)) 

1.0

## NerDLModel

### Public NER

Available Entities are : PERSON, LOCATION, ORGANIZATION, MISC

In [0]:
public_ner = NerDLModel.pretrained("ner_dl", 'en') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# ner_dl model is trained with glove_100d. So we use the same embeddings in the pipeline
glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d').\
  setInputCols(["document", 'token']).\
  setOutputCol("embeddings")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 glove_embeddings,
 public_ner
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [0]:
result = nlpPipelineModel.transform(news_df.limit(100))

In [0]:
result = pipelineModel.transform(news_df.limit(100))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias('cols'))\
.select(F.expr("cols['0']").alias('token'),
        F.expr("cols['1']").alias('ner_label'))

result_df.show(30, truncate=100)

+------------+---------+
|       token|ner_label|
+------------+---------+
|      Unions|        O|
|representing|        O|
|     workers|        O|
|          at|        O|
|      Turner|    B-ORG|
|      Newall|    I-ORG|
|         say|        O|
|        they|        O|
|         are|        O|
|           '|        O|
|disappointed|        O|
|           '|        O|
|       after|        O|
|       talks|        O|
|        with|        O|
|    stricken|        O|
|      parent|        O|
|        firm|        O|
|     Federal|    B-ORG|
|       Mogul|    I-ORG|
|           .|        O|
|     TORONTO|    B-LOC|
|           ,|        O|
|      Canada|    B-LOC|
|           A|        O|
|      second|        O|
|        team|        O|
|          of|        O|
|  rocketeers|        O|
|   competing|        O|
+------------+---------+
only showing top 30 rows



## NerDL OntoNotes 100D

Available Entities: 'CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART'

In [0]:
onto_ner = NerDLModel.pretrained("onto_100", 'en') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 glove_embeddings,
 onto_ner
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

onto_100 download started this may take some time.
Approximate size to download 13.5 MB
[OK!]


In [0]:
result = pipelineModel.transform(news_df.limit(10))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))

result_df.show(100, truncate=100)

+------------+-------------+
|       token|    ner_label|
+------------+-------------+
|      Unions|            O|
|representing|            O|
|     workers|            O|
|          at|            O|
|      Turner|        B-ORG|
|      Newall|        I-ORG|
|         say|            O|
|        they|            O|
|         are|            O|
|           '|            O|
|disappointed|            O|
|           '|            O|
|       after|            O|
|       talks|            O|
|        with|            O|
|    stricken|            O|
|      parent|            O|
|        firm|            O|
|     Federal|        B-ORG|
|       Mogul|        I-ORG|
|           .|            O|
|     TORONTO|        B-GPE|
|           ,|            O|
|      Canada|        B-GPE|
|           A|            O|
|      second|    B-ORDINAL|
|        team|            O|
|          of|            O|
|  rocketeers|            O|
|   competing|            O|
|         for|            O|
|         the|

## Getting the NER Chunks with NER Converter

In [0]:

ner_converter = NerConverter() \
  .setInputCols(["document", "token", "ner"]) \
  .setOutputCol("ner_chunk")


nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 glove_embeddings,
 onto_ner,
 ner_converter
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(news_df.limit(10))

In [0]:
result = pipelineModel.transform(news_df.limit(10))


result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------------+-----------+
|chunk                              |ner_label  |
+-----------------------------------+-----------+
|Turner   Newall                    |ORG        |
|Federal Mogul                      |ORG        |
|TORONTO                            |GPE        |
|Canada                             |GPE        |
|second                             |ORDINAL    |
|#36;10 million                     |CARDINAL   |
|Ansari X Prize                     |WORK_OF_ART|
|first                              |ORDINAL    |
|the University of Louisville       |ORG        |
|Mike Fitzpatrick                   |PERSON     |
|the day                            |DATE       |
|Southern California's              |LOC        |
|Friday                             |DATE       |
|first                              |ORDINAL    |
|British                            |NORP       |
|Department for Education and Skills|ORG        |
|""Music Manifesto""                |WORK_OF_ART|


In [0]:
# To access to metadata which we will use to get entity, we have to use fullannotate rather tahn annotate

light_model = LightPipeline(pipelineModel)

light_result = light_model.fullAnnotate('Unions representing workers at Turner Newall say they are disappointed after talks with stricken parent firm Federal Mogul and Mike Fitzpatrick in Canada.')

In [0]:
chunks = []
entities  = []

for n in light_result[0]['ner_chunk']:
    chunks.append(n.result)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({"chunks":chunks, 'entities':entities})
df

,chunks,entities
0,Turner Newall,ORG
1,Federal Mogul,ORG
2,Mike Fitzpatrick,PERSON
3,Canada,GPE


## Highlight Entities

In [0]:
import random
from IPython.core.display import display, HTML


def get_color():
    r = lambda: random.randint(100,255)
    return '#%02X%02X%02X' % (r(),r(),r())

In [0]:
from spacy import displacy

def show_html_spacy(annotated_text, filter_labels=True):

    label_list = []
    sent_dict_list = []
    
    for n in annotated_text['ner_chunk']:

        ent = {'start': n.begin, 'end':n.end+1, 'label':n.metadata['entity'].upper()}
        
        label_list.append(n.metadata['entity'].upper())

        sent_dict_list.append(ent)
   
    document_text = [{'text':annotated_text['document'][0].result, 'ents':sent_dict_list,'title':None}]

    label_list = list(set(label_list))
                
    label_color={}
    
    for l in label_list:
        
        label_color[l]=get_color()
    
    colors = {k:label_color[k] for k in label_list}
        
    
    html_text = displacy.render(document_text, style='ent', jupyter=True, manual=True, options= {"ents": label_list, 'colors': colors})

    return html_text

In [0]:
ann_text = light_model.fullAnnotate('Unions representing workers at Turner Newall say they are disappointed after talks with stricken parent firm Federal Mogul and Mike Fitzpatrick in Canada.')

show_html_spacy (ann_text[0])

In [0]:
def get_NER_html (annotated_text):
    
    light_data=annotated_text

    html_output=''
    
    problem_flag = False
    new_problem = []
    problem_list = []
    
    label_list = list(set([i.split('-')[1] for i in light_data['ner'] if i!='O']))
    
        
    label_color={}
    
    for l in label_list:
        
        label_color[l]=get_color()
            
    for index, this_token in enumerate(light_data['token']):

        try:
            ent = light_data['ner'][index].split('-')[1]
        except:
            ent = light_data['ner'][index]
        
       
        if ent in label_list:
            color = label_color[ent]
            html_output+='<SPAN style="background-color: {}">'.format(color) + this_token + " </SPAN>"
        else:
            html_output+=this_token + " "
        

    html_output += '</div>'
    html_output += '<div>Color codes:'
    

    for l in label_list:
        
        html_output += '<SPAN style="background-color: {}">{}</SPAN>, '.format(label_color[l],l)
   
    
    return display(HTML(html_output))

In [0]:

ann_text = light_model.annotate('Unions representing workers at Turner Newall say they are disappointed after talks with stricken parent firm Federal Mogul and Mike Fitzpatrick in Canada.')

get_NER_html (ann_text)

## BELOW CODES IS BELOGS TO Training02 and Training01 Respectively

In [0]:
!wget https://gist.githubusercontent.com/vkocaman/e091605f012ffc1efc0fcda170919602/raw/fae33d25bd026375b2aaf1194b68b9da559c4ac4/annotators.csv

--2020-04-20 13:06:50--  https://gist.githubusercontent.com/vkocaman/e091605f012ffc1efc0fcda170919602/raw/fae33d25bd026375b2aaf1194b68b9da559c4ac4/annotators.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3136 (3.1K) [text/plain]
Saving to: ‘annotators.csv’

annotators.csv      100%[===================>]   3.06K  --.-KB/s    in 0s      

2020-04-20 13:06:50 (49.3 MB/s) - ‘annotators.csv’ saved [3136/3136]



In [0]:
import pandas as pd

df = pd.read_csv('annotators.csv')

In [0]:
df

,Annotator,Description,Version,Annotator Approach,Annotator Model
0,Tokenizer*,Identifies tokens with tokenization open stand...,Opensource,-,+
1,Normalizer*,Removes all dirty characters from text,Opensource,-,+
2,Stemmer*,Returns hard'-stems out of words with the obje...,Opensource,+,-
3,Lemmatizer*,Retrieves lemmas out of words with the objecti...,Opensource,-,+
4,RegexMatcher*,Uses a reference file to match a set of regula...,Opensource,+,+
5,TextMatcher*,Annotator to match entire phrases (by token) p...,Opensource,+,+
6,Chunker*,Matches a pattern of part'-of'-speech tags in ...,Opensource,+,-
7,DateMatcher*,Reads from different forms of date and time ex...,Opensource,+,-
8,SentenceDetector*,Finds sentence bounds in raw text. Applies rul...,Opensource,+,-
9,DeepSentenceDetector*,Finds sentence bounds in raw text. Applies a N...,Opensource,+,-


In [0]:
text = 'Peter Parker is a nice guy and lives in New York.'

# spark is our session varaible we have created above as 'spark' = sparknlp.start() 
spark_df = spark.createDataFrame(([[text]])).toDF('text')
spark_df.show(truncate=False)

+-------------------------------------------------+
|text                                             |
+-------------------------------------------------+
|Peter Parker is a nice guy and lives in New York.|
+-------------------------------------------------+



In [0]:
spark_df = spark.createDataFrame(df)
spark_df.show(truncate=False)

+-----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+---------------+
|Annotator              |Description                                                                                                                                                                                                |Version   |Annotator Approach|Annotator Model|
+-----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+---------------+
|Tokenizer*             |Identifies tokens with tokenization open standards                                                                                                 

In [0]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/spark-nlp-basics/sample-sentences-en.txt

--2020-04-20 13:21:01--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/spark-nlp-basics/sample-sentences-en.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284 [text/plain]
Saving to: ‘sample-sentences-en.txt’

sample-sentences-en 100%[===================>]     284  --.-KB/s    in 0s      

2020-04-20 13:21:01 (17.7 MB/s) - ‘sample-sentences-en.txt’ saved [284/284]



In [0]:
# When we use read.text --> it reads line by line
spark_df1 = spark.read.text('./sample-sentences-en.txt').toDF('text')
spark_df1.show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [0]:
# Eger sadece 1 column secmek istersek
spark_df1.select('text').show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [0]:
# The wholeTextFiles reads file as whole single document not line by line
textFiles = spark.sparkContext.wholeTextFiles("./*.txt", 4)
spark_df = textFiles.toDF(schema=['path', 'file_content'])
spark_df.show(truncate=30)

+------------------------------+------------------------------+
|                          path|                  file_content|
+------------------------------+------------------------------+
|file:/content/sample-senten...|Peter is a very good person...|
+------------------------------+------------------------------+



In [0]:
spark_df.select('file_content').take(1)

[Row(file_content="Peter is a very good person.\nMy life in Russia is very interesting.\nJohn and Peter are brothers. However they don't support each other that much.\nLucas Nogal Dunbercker is no longer happy. He has a good car though.\nEurope is very culture rich. There are huge churches! and big houses!")]

## Document Assambler

In [0]:
from sparknlp.base import *

In [0]:
documentAssembler = DocumentAssembler()\
.setInputCol('text')\
.setOutputCol('document')\
.setCleanupMode('shrink') # bosluklari, tab'leri vs temizler

doc_df = documentAssembler.transform(spark_df1)
doc_df.show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|text                                                                         |document                                                                                                               |
+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|Peter is a very good person.                                                 |[[document, 0, 27, Peter is a very good person., [sentence -> 0], []]]                                                 |
|My life in Russia is very interesting.                                       |[[document, 0, 37, My life in Russia is very interesting., [sentence -> 0], []]]                                       |


In [0]:
doc_df.printSchema()
# as we can see we have the metadata here

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [0]:
doc_df.select('document.result', 'document.begin', 'document.end').show(truncate=False)
# each one returns a list type

+-------------------------------------------------------------------------------+-----+----+
|result                                                                         |begin|end |
+-------------------------------------------------------------------------------+-----+----+
|[Peter is a very good person.]                                                 |[0]  |[27]|
|[My life in Russia is very interesting.]                                       |[0]  |[37]|
|[John and Peter are brothers. However they don't support each other that much.]|[0]  |[76]|
|[Lucas Nogal Dunbercker is no longer happy. He has a good car though.]         |[0]  |[67]|
|[Europe is very culture rich. There are huge churches! and big houses!]        |[0]  |[68]|
+-------------------------------------------------------------------------------+-----+----+



In [0]:
# esas df'te document column uzerinde islem yapiyoruz o yuzden document.result olarak tanimlamaliyiz
doc_df.select("document.result").take(1)

[Row(result=['Peter is a very good person.'])]

In [0]:
### Expoloding teh dataframe (to view in much better nice view)

import pyspark.sql.functions as F

doc_df.withColumn(
    "tmp",
    F.explode('document'))\
    .select('tmp.*')\
    .show(truncate=False)

+-------------+-----+---+-----------------------------------------------------------------------------+---------------+----------+
|annotatorType|begin|end|result                                                                       |metadata       |embeddings|
+-------------+-----+---+-----------------------------------------------------------------------------+---------------+----------+
|document     |0    |27 |Peter is a very good person.                                                 |[sentence -> 0]|[]        |
|document     |0    |37 |My life in Russia is very interesting.                                       |[sentence -> 0]|[]        |
|document     |0    |76 |John and Peter are brothers. However they don't support each other that much.|[sentence -> 0]|[]        |
|document     |0    |67 |Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |[sentence -> 0]|[]        |
|document     |0    |68 |Europe is very culture rich. There are huge churches! and 

## Sentence Detector

In [0]:
from sparknlp.annotator import *

# we feed the document column coming from Document Assembler

sentenceDetector = SentenceDetector().\
setInputCols(['document']).\
setOutputCol('sentences')

In [0]:
sent_df = sentenceDetector.transform(doc_df)
sent_df.show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                         |document                                                                                                               |sentences                                                                                                                                                                                          |
+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+---------

In [0]:
# take works as head in pandas
sent_df.select('sentences').take(3)

[Row(sentences=[Row(annotatorType='document', begin=0, end=27, result='Peter is a very good person.', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentences=[Row(annotatorType='document', begin=0, end=37, result='My life in Russia is very interesting.', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentences=[Row(annotatorType='document', begin=0, end=27, result='John and Peter are brothers.', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='document', begin=29, end=76, result="However they don't support each other that much.", metadata={'sentence': '1'}, embeddings=[])])]

In [0]:
sample_text ="""The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology 
service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro 
with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should 
be discontinued indefinitely fro 3 months ."""
sample_text

'The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology \nservice and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro \nwith meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should \nbe discontinued indefinitely fro 3 months .'

In [0]:
spark_df2 = spark.createDataFrame([[sample_text]]).toDF('text')
spark_df2.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                   |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
doc_df1 = documentAssembler.transform(spark_df2)
sent_df = sentenceDetector.transform(doc_df1)
sent_df.show(truncate=40)

+----------------------------------------+----------------------------------------+----------------------------------------+
|                                    text|                                document|                               sentences|
+----------------------------------------+----------------------------------------+----------------------------------------+
|The patient was prescribed 1 capsule ...|[[document, 0, 339, The patient was p...|[[document, 0, 57, The patient was pr...|
+----------------------------------------+----------------------------------------+----------------------------------------+



In [0]:
sent_df.select('sentences.result').take(1)

[Row(result=['The patient was prescribed 1 capsule of Advil for 5 days .', 'He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .', 'It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'])]

In [0]:
sentenceDetector.setExplodeSentences(True)

SentenceDetector_18a10300d4df

In [0]:
sent_df = sentenceDetector.transform(doc_df1)
sent_df.select('sentences.result').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                      |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[The patient was prescribed 1 capsule of Advil for 5 days .]                                                                                                                                |
|[He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .]|
|[It was determined that all SGLT2 inhibitors

## Tokenizer

In [0]:
# #we can also use sentence in InputCols
tokenizer = Tokenizer()\
            .setInputCols(['document'])\
            .setOutputCol('token')

In [0]:
text2 = "Peter Parker (Spiderman) is a nice guy and live ins New York but has no e-mail! and he's gone so far"
text2.split()
# There are unwanted chars below which we do not want a part fo token such as () and !

['Peter',
 'Parker',
 '(Spiderman)',
 'is',
 'a',
 'nice',
 'guy',
 'and',
 'live',
 'ins',
 'New',
 'York',
 'but',
 'has',
 'no',
 'e-mail!',
 'and',
 "he's",
 'gone',
 'so',
 'far']

In [0]:
spark_df = spark.createDataFrame([[text2]]).toDF('text')

In [0]:
doc_df = documentAssembler.transform(spark_df)
token_df = tokenizer.fit(doc_df).transform(doc_df)
token_df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                                text|                                                                                            document|                                                                                               token|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|Peter Parker (Spiderman) is a nice guy and live ins New York but has no e-mail! and 

In [0]:
token_df.select('token.result').take(1)

[Row(result=['Peter', 'Parker', '(', 'Spiderman', ')', 'is', 'a', 'nice', 'guy', 'and', 'live', 'ins', 'New', 'York', 'but', 'has', 'no', 'e-mail', '!', 'and', "he's", 'gone', 'so', 'far'])]

In [0]:
# Eger tokenizer i custome etmek istersek

tokenizer = Tokenizer()\
            .setInputCols(['document'])\
            .setOutputCol('token')\
            .setSplitChars(['-'])\
            .setContextChars(['(', ')', '?', '!'])\
            .addException('New York')

In [0]:
token_df = tokenizer.fit(doc_df).transform(token_df)
token_df.select('token.result').take(1)

[Row(result=['Peter', 'Parker', '(', 'Spiderman', ')', 'is', 'a', 'nice', 'guy', 'and', 'live', 'ins', 'New York', 'but', 'has', 'no', 'e', 'mail', '!', 'and', "he's", 'gone', 'so', 'far'])]

In [0]:
# # Pay Attention to () for Spiderman
tokenizer33 = Tokenizer()\
            .setInputCols(['document'])\
            .setOutputCol('token')\
            .setSplitChars(['-'])\
            .setContextChars(['?', '!'])\
            .addException('New York')

In [0]:
token_df = tokenizer33.fit(doc_df).transform(token_df)
token_df.select('token.result').take(1)

[Row(result=['Peter', 'Parker', '(Spiderman)', 'is', 'a', 'nice', 'guy', 'and', 'live', 'ins', 'New York', 'but', 'has', 'no', 'e', 'mail', '!', 'and', "he's", 'gone', 'so', 'far'])]

In [0]:
### Let's Combine Everything in a Pipeline

In [0]:
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
                    .setInputCol('text')\
                    .setOutputCol('document')

sentenceDetector = SentenceDetector()\
                    .setInputCols(['document'])\
                    .setOutputCol('sentences')

tokenizer = Tokenizer()\
            .setInputCols(['sentences'])\
            .setOutputCol('token')
        
nlPipeline = Pipeline(stages=[
                              documentAssembler,
                              sentenceDetector,
                              tokenizer
])

empty_df = spark.createDataFrame([['']]).toDF('text')

pipelineModel = nlPipeline.fit(empty_df)

In [0]:
spark_df = spark.read.text('./sample-sentences-en.txt').toDF('text')
spark_df.show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [0]:
result = pipelineModel.transform(spark_df)
result.show(truncate=20)

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|           sentences|               token|
+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[document, 0, 27...|[[token, 0, 4, Pe...|
|My life in Russia...|[[document, 0, 37...|[[document, 0, 37...|[[token, 0, 1, My...|
|John and Peter ar...|[[document, 0, 76...|[[document, 0, 27...|[[token, 0, 3, Jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[document, 0, 41...|[[token, 0, 4, Lu...|
|Europe is very cu...|[[document, 0, 68...|[[document, 0, 27...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+



In [0]:
result.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentences: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = tru

In [0]:
result.select('sentences.result').take(3)

[Row(result=['Peter is a very good person.']),
 Row(result=['My life in Russia is very interesting.']),
 Row(result=['John and Peter are brothers.', "However they don't support each other that much."])]

In [0]:
result.select('token').take(3)[2]

Row(token=[Row(annotatorType='token', begin=0, end=3, result='John', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=5, end=7, result='and', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=9, end=13, result='Peter', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=15, end=17, result='are', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=19, end=26, result='brothers', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=27, end=27, result='.', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=29, end=35, result='However', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=37, end=40, result='they', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=42, end=46, result="don't", metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=48, end=54, result='s

In [0]:
result.select('token.result').take(3)

[Row(result=['Peter', 'is', 'a', 'very', 'good', 'person', '.']),
 Row(result=['My', 'life', 'in', 'Russia', 'is', 'very', 'interesting', '.']),
 Row(result=['John', 'and', 'Peter', 'are', 'brothers', '.', 'However', 'they', "don't", 'support', 'each', 'other', 'that', 'much', '.'])]

In [0]:
''.join([i for i in "My name is @# Serdar." if i.isalnum() or i==' '])

'My name is  Serdar'

## Normalizer

In [0]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
from sparknlp.annotator import *

normalizer = Normalizer()\
            .setInputCols(['token'])\
            .setOutputCol('normalized')\
            .setLowercase(True)\
            .setCleanupPatterns(["[^\w\d\s]"]) #remove punctuations and keep alphanumerics only
            # if we do not set ClenaupPatterns it till only keep alphabet letters [^A-Za-z]

In [0]:
# We're removing sentence annotator


documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlPipeline = Pipeline(stages=[
                              documentAssembler,
                              tokenizer,
                              normalizer
])
empty_df = spark.createDataFrame([['']]).toDF('text')

pipelineModel = nlPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(spark_df)
result.show(truncate=True)

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|          normalized|
+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|
|My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|
|John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|
|Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|
+--------------------+--------------------+--------------------+--------------------+



In [0]:
result.select('normalized.result').take(3)

[Row(result=['peter', 'is', 'a', 'very', 'good', 'person']),
 Row(result=['my', 'life', 'in', 'russia', 'is', 'very', 'interesting']),
 Row(result=['john', 'and', 'peter', 'are', 'brothers', 'however', 'they', 'dont', 'support', 'each', 'other', 'that', 'much'])]

## Stopwrods

In [0]:
stopwords_cleaner = StopWordsCleaner()\
                    .setInputCols('token')\
                    .setOutputCol('cleanTokens')\
                    .setCaseSensitive(False)
                    #.setStopWords(['no', 'without']) custom stopwords we can use but it will overwrite existing stopwords and will ONLY use this one
                    # if we want to  use normalized ones and just want to use from normalised use "normalized" in InputCols

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlPipeline = Pipeline(stages=[
                              documentAssembler,
                              tokenizer,
                              stopwords_cleaner
])

empty_df = spark.createDataFrame([['']]).toDF('text')

pipelineModel = nlPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(spark_df)
result.show(truncate=True)

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|         cleanTokens|
+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|
|My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 3, 6, li...|
|John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, Jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, Lu...|
|Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+



In [0]:
result.select('cleanTokens.result').take(3)
# We did not include Normalizer so we might see punctuations and uppercase. If we dont want, se should incluce in pipeline

[Row(result=['Peter', 'good', 'person', '.']),
 Row(result=['life', 'Russia', 'interesting', '.']),
 Row(result=['John', 'Peter', 'brothers', '.', 'However', 'support', 'much', '.'])]

## Token Assembler

In [0]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

sentenceDetector = SentenceDetector().\
    setInputCols(['document']).\
    setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(False)\

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)\

tokenassembler = TokenAssembler()\
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("clean_text")


nlpPipeline = Pipeline(stages=[
     documentAssembler,
    sentenceDetector,
     tokenizer,
     normalizer,
     stopwords_cleaner,
     tokenassembler
 ])


empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|           sentences|               token|          normalized|         cleanTokens|          clean_text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|[[document, 0, 16...|
|My life in Russia...|[[document, 0, 37...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, My...|[[token, 3, 6, li...|[[document, 3, 25...|
|John and Peter ar...|[[document, 0, 76...|[[document, 0, 27...|[[token, 0, 3, Jo...|[[token, 0, 3, Jo...|[[token, 0, 3, Jo...|[[document, 0, 18...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[document, 0, 41...|[[token, 0, 4, Lu...|[[token, 0, 4, Lu...|

In [0]:
result.select(['text', 'clean_text.result']).show(truncate=False)

+-----------------------------------------------------------------------------+------------------------------------------------------+
|text                                                                         |result                                                |
+-----------------------------------------------------------------------------+------------------------------------------------------+
|Peter is a very good person.                                                 |[Peter good person]                                   |
|My life in Russia is very interesting.                                       |[life Russia interesting]                             |
|John and Peter are brothers. However they don't support each other that much.|[John Peter brothers, However dont support much]      |
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |[Lucas Nogal Dunbercker longer happy, good car though]|
|Europe is very culture rich. There are huge churches! 

In [0]:
import pyspark.sql.functions as F

result.withColumn(
    "tmp",
    F.explode('clean_text'))\
    .select("tmp.*").select('begin', 'end', 'result', 'metadata.sentence').show(truncate=False)

+-----+---+-----------------------------------+--------+
|begin|end|result                             |sentence|
+-----+---+-----------------------------------+--------+
|0    |16 |Peter good person                  |0       |
|3    |25 |life Russia interesting            |0       |
|0    |18 |John Peter brothers                |0       |
|29   |53 |However dont support much          |1       |
|0    |34 |Lucas Nogal Dunbercker longer happy|0       |
|52   |66 |good car though                    |1       |
|0    |18 |Europe culture rich                |0       |
|39   |51 |huge churches                      |1       |
|58   |67 |big houses                         |2       |
+-----+---+-----------------------------------+--------+



In [0]:
# if we hadn't used Sentence Detector, this would be what we got. (tokenizer gets document instead of sentences column)

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")


nlpPipeline = Pipeline(stages=[
     documentAssembler,
     tokenizer,
     normalizer,
     stopwords_cleaner,
     tokenassembler
 ])


empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(spark_df)

result.select('text', 'clean_text.result').show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------+
|text                                                                         |result                                               |
+-----------------------------------------------------------------------------+-----------------------------------------------------+
|Peter is a very good person.                                                 |[Peter good person]                                  |
|My life in Russia is very interesting.                                       |[life Russia interesting]                            |
|John and Peter are brothers. However they don't support each other that much.|[John Peter brothers However dont support much]      |
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |[Lucas Nogal Dunbercker longer happy good car though]|
|Europe is very culture rich. There are huge churches! and big

In [0]:
result.withColumn(
    "tmp",
    F.explode('clean_text'))\
    .select("tmp.*").select('begin', 'end', 'result', 'metadata.sentence').show(truncate=False)

# We get all sentences in a single lien and all sentences assigend as 0, there si no other sentence

+-----+---+---------------------------------------------------+--------+
|begin|end|result                                             |sentence|
+-----+---+---------------------------------------------------+--------+
|0    |16 |Peter good person                                  |0       |
|3    |25 |life Russia interesting                            |0       |
|0    |44 |John Peter brothers However dont support much      |0       |
|0    |50 |Lucas Nogal Dunbercker longer happy good car though|0       |
|0    |43 |Europe culture rich huge churches big houses       |0       |
+-----+---+---------------------------------------------------+--------+



## Stemmer

In [0]:
stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

In [0]:

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:

result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|
+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|
|My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|
|John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|
|Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|
+--------------------+--------------------+--------------------+--------------------+



In [0]:
result.select('stem.result').show(truncate=False)

+-------------------------------------------------------------------------------------------+
|result                                                                                     |
+-------------------------------------------------------------------------------------------+
|[peter, i, a, veri, good, person, .]                                                       |
|[my, life, in, russia, i, veri, interest, .]                                               |
|[john, and, peter, ar, brother, ., howev, thei, don't, support, each, other, that, much, .]|
|[luca, nogal, dunberck, i, no, longer, happi, ., he, ha, a, good, car, though, .]          |
|[europ, i, veri, cultur, rich, ., there, ar, huge, church, !, and, big, hous, !]           |
+-------------------------------------------------------------------------------------------+



In [0]:
import pyspark.sql.functions as F

result_df = result.select(F.explode(F.arrays_zip('token.result', 'stem.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("stem")).toPandas()

result_df.head(10)

,token,stem
0,Peter,peter
1,is,i
2,a,a
3,very,veri
4,good,good
5,person,person
6,.,.
7,My,my
8,life,life
9,in,in


## Lemmatizer

In [0]:
!wget https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt

--2020-04-20 16:01:21--  https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1348552 (1.3M) [text/plain]
Saving to: ‘AntBNC_lemmas_ver_001.txt’

AntBNC_lemmas_ver_0 100%[===================>]   1.29M  --.-KB/s    in 0.01s   

2020-04-20 16:01:21 (128 MB/s) - ‘AntBNC_lemmas_ver_001.txt’ saved [1348552/1348552]



In [0]:
lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

In [0]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|               lemma|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|[[token, 0, 4, Pe...|
|My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|[[token, 0, 1, My...|
|John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|[[token, 0, 3, Jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|[[token, 0, 4, Lu...|
|Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [0]:
result.select('lemma.result').show(truncate=False)

+---------------------------------------------------------------------------------------------+
|result                                                                                       |
+---------------------------------------------------------------------------------------------+
|[Peter, be, a, very, good, person, .]                                                        |
|[My, life, in, Russia, be, very, interest, .]                                                |
|[John, and, Peter, be, brother, ., However, they, don't, support, each, other, that, much, .]|
|[Lucas, Nogal, Dunbercker, be, no, long, happy, ., He, have, a, good, car, though, .]        |
|[Europe, be, very, culture, rich, ., There, be, huge, church, !, and, big, house, !]         |
+---------------------------------------------------------------------------------------------+



In [0]:

result_df = result.select(F.explode(F.arrays_zip('token.result', 'stem.result',  'lemma.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("stem"),
        F.expr("cols['2']").alias("lemma")).toPandas()

result_df.head(10)

,token,stem,lemma
0,Peter,peter,Peter
1,is,i,be
2,a,a,a
3,very,veri,very
4,good,good,good
5,person,person,person
6,.,.,.
7,My,my,My
8,life,life,life
9,in,in,in


## N-Gram

In [0]:
ngrams_cum = NGramGenerator() \
            .setInputCols(["token"]) \
            .setOutputCol("ngrams") \
            .setN(3) \
            .setEnableCumulative(True)\
            .setDelimiter("_") # Default is space
    
# .setN(3) means, take bigrams and trigrams.

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 ngrams_cum
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(spark_df)

result.select('ngrams.result').show(truncate=200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                  result|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                    [Peter, is, a, very, good, person, ., Peter_is, is_a, a_very, very_good, good_person, person_., Peter_is_a, is_a_very, a_very_good, very_good_person, good_person_.]|
|[My, life, in, Russia, is, very, interesting, ., My_life, life_in, in_Russia, Russia_is, is_very, very_interesting, interesting_., My_life_in, life_in_Russia, in_Russia_is, Russia_is_very

## Text Matcher

In [0]:
! wget 	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv

--2020-04-20 16:11:48--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.110.13
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.110.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10484510 (10.0M) [text/csv]
Saving to: ‘pubmed-sample.csv’

pubmed-sample.csv   100%[===================>]  10.00M  16.8MB/s    in 0.6s    

2020-04-20 16:11:49 (16.8 MB/s) - ‘pubmed-sample.csv’ saved [10484510/10484510]



In [0]:
!head pubmed-sample.csv

AB,TI
"The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle. The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.","Genomic structure and expression 

In [0]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("./pubmed-sample.csv")\
                .filter("AB IS NOT null")\
                .withColumnRenamed("AB", "text")\
                .drop("TI")

pubMedDF.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potas...|
|BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer...|
|OBJECTIVE: To investigate the relationship between preoperative atrialfibrillation and early and ...|
|Combined EEG/fMRI recording has been used to localize the generators of EEGevents and to identify...|
|Kohlschutter syndrome is a rare neurodegenerative disorder presenting withintractable seizures, d...|
|Statistical analysis of neuroimages is commonly approached with intergroupcomparisons made by rep...|
|The synthetic DOX-LNA conjugate was characterized by proton nuclear magn

In [0]:
# let's create a file which will includes the entities we want to find
# it will be stored in content directory
# It is used for the EXACT WORD/WORDS we're looking for

entities = ['KCNJ9', 'GIRK', 'diabates mellitus', 'nucleotide polymorphisms']

with open('clinical_entities.txt', 'w') as f:
    for i in entities:
        f.write(i+'\n')

In [0]:
entity_extractor = TextMatcher()\
                    .setInputCols(['document', 'token'])\
                    .setOutputCol('matched_entities')\
                    .setEntities('clinical_entities.txt')

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 entity_extractor
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
pubMedDF.count()
# it has 7.5K rows

7537

In [0]:
result = pipelineModel.transform(pubMedDF.limit(10))

In [0]:
pubMedDF.select('text').take(1)

[Row(text='The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle. The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.')]

In [0]:
result.select('matched_entities.result').take(1)

[Row(result=['KCNJ9', 'GIRK', 'KCNJ9', 'nucleotide polymorphisms', 'KCNJ9', 'KCNJ9'])]

In [0]:
result_df = result.select(F.explode(F.arrays_zip('matched_entities.result', 'matched_entities.begin',  'matched_entities.end')).alias("cols")) \
.select(F.expr("cols['0']").alias("matched_entities"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end")).toPandas()

result_df.head(10)

,matched_entities,begin,end
0,KCNJ9,10,14
1,GIRK,103,106
2,KCNJ9,173,177
3,nucleotide polymorphisms,471,494
4,KCNJ9,801,805
5,KCNJ9,868,872


## RegexMatcher

In [0]:
rules = '''
renal\s\w+, followed by 'renal'
cardiac\s\w+, followed by 'cardiac'
'''

with open('regex_rules.txt', 'w') as f:
    
    f.write(rules)

In [0]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

regex_matcher = RegexMatcher()\
    .setInputCols('document')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_matches")\
    .setExternalRules(path='./regex_rules.txt', delimiter=',')
    

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 regex_matcher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(pubMedDF)

match_df.select('regex_matches.result').take(3)

[Row(result=[]),
 Row(result=[]),
 Row(result=['renal failure', 'cardiac surgery', 'cardiac surgery', 'cardiac surgical'])]

In [0]:
# Sadece regex iceren satirlari gormek istersek

match_df.select('text','regex_matches.result')\
.toDF('text','matches').filter(F.size('matches')>1)\
.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                                           matches|
+--------------------------------------------------+--------------------------------------------------+
|OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
|Aging is a physiological process that causes st...|                     [renal function, renal aging]|
|BACKGROUND: Left-sided inferior vena cava (IVC)...|          [renal surgery, renal mass, renal veins]|
|A successful kidney transplantation (Tx) offers...|                         [renal failure, renal Tx]|
|The transcription factor scleraxis has been imp...|[cardiac valves, cardiac fibroblasts, cardiac c...|
|Historically, the only surgical option for remo...|                        [renal mass, renal masses]|
|OBJECTIVES: To examine the clinical effectivene...|   [renal tr

In [0]:
match_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- regex_matches: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull =

## Finisher

In [0]:
# It removes previous columns which is defined in pipeline and shows only the one which is defined in finisher
# Finisher: Once we have our NLP pipeline ready to go, we might want to use our annotation results somewhere else where it is easy to use. 
# The Finisher outputs annotation(s) values into a string.

In [0]:
finisher = Finisher()\
        .setInputCols(['regex_matches'])\
        .setIncludeMetadata(False) # set False to remove metadata

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 regex_matcher,
 finisher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(pubMedDF)

match_df.show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                            finished_regex_matches|
+--------------------------------------------------+--------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|                                                []|
|BACKGROUND: At present, it is one of the most i...|                                                []|
|OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
|Combined EEG/fMRI recording has been used to lo...|                                                []|
|Kohlschutter syndrome is a rare neurodegenerati...|                                                []|
|Statistical analysis of neuroimages is commonly...|                                                []|
|The synthetic DOX-LNA conjugate was characteriz...|            

In [0]:
match_df.printSchema()
# it is very simple now

root
 |-- text: string (nullable = true)
 |-- finished_regex_matches: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
# If we want to filter out the empty matches

match_df.filter(F.size('finished_regex_matches')>=1).show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                            finished_regex_matches|
+--------------------------------------------------+--------------------------------------------------+
|OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
|Successful renal transplantation is associated ...|                           [renal transplantation]|
|Although inhaled and intranasal corticosteroids...|                               [renal suppression]|
|Aging is a physiological process that causes st...|                     [renal function, renal aging]|
|Tight junctions form the major paracellular bar...|                                     [renal tight]|
|BACKGROUND: Left-sided inferior vena cava (IVC)...|          [renal surgery, renal mass, renal veins]|
|A successful kidney transplantation (Tx) offers...|            

## LightPipeline

LightPipelines are Spark NLP specific Pipelines, equivalent to Spark ML Pipeline, but meant to **deal with smaller amounts of data**. They’re useful working with small datasets, debugging results, or when running either training or prediction from an API that serves one-off requests.

Spark NLP LightPipelines are Spark ML pipelines converted into a single machine but the multi-threaded task, becoming more than 10x times faster for smaller amounts of data (small is relative, but 50k sentences are roughly a good maximum). To use them, we simply plug in a trained (fitted) pipeline and then annotate a plain text. We don't even need to convert the input text to DataFrame in order to feed it into a pipeline that's accepting DataFrame as an input in the first place. This feature would be quite useful when it comes to getting a prediction for a few lines of text from a trained ML model.

It is nearly 20x faster than using Spark ML Pipeline

In [0]:
# Note we re-run the pipelineModel which inludes 'lemma', 'stem' etc above so the the very last one we're processing which is coming 
# from regex_matches. Please pya attention that

In [0]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate("John and Peter are brothers. However they don't support each other that much")

In [0]:
light_result.keys()

dict_keys(['document', 'token', 'stem', 'lemma'])

In [0]:
list(zip(light_result['token'], light_result['stem'], light_result['lemma']))

[('John', 'john', 'John'),
 ('and', 'and', 'and'),
 ('Peter', 'peter', 'Peter'),
 ('are', 'ar', 'be'),
 ('brothers', 'brother', 'brother'),
 ('.', '.', '.'),
 ('However', 'howev', 'However'),
 ('they', 'thei', 'they'),
 ("don't", "don't", "don't"),
 ('support', 'support', 'support'),
 ('each', 'each', 'each'),
 ('other', 'other', 'other'),
 ('that', 'that', 'that'),
 ('much', 'much', 'much')]

In [0]:
light_result = light_model.fullAnnotate("John and Peter are brothers. However they don't support each other that much")
light_result

[{'document': [Annotation(document, 0, 75, John and Peter are brothers. However they don't support each other that much, {})],
  'lemma': [Annotation(token, 0, 3, John, {'sentence': '0'}),
   Annotation(token, 5, 7, and, {'sentence': '0'}),
   Annotation(token, 9, 13, Peter, {'sentence': '0'}),
   Annotation(token, 15, 17, be, {'sentence': '0'}),
   Annotation(token, 19, 26, brother, {'sentence': '0'}),
   Annotation(token, 27, 27, ., {'sentence': '0'}),
   Annotation(token, 29, 35, However, {'sentence': '0'}),
   Annotation(token, 37, 40, they, {'sentence': '0'}),
   Annotation(token, 42, 46, don't, {'sentence': '0'}),
   Annotation(token, 48, 54, support, {'sentence': '0'}),
   Annotation(token, 56, 59, each, {'sentence': '0'}),
   Annotation(token, 61, 65, other, {'sentence': '0'}),
   Annotation(token, 67, 70, that, {'sentence': '0'}),
   Annotation(token, 72, 75, much, {'sentence': '0'})],
  'stem': [Annotation(token, 0, 3, john, {'sentence': '0'}),
   Annotation(token, 5, 7, and,

In [0]:
### First NoteBook Can Be Seen From Here to Below

## Using Pretrained Pipelines

https://github.com/JohnSnowLabs/spark-nlp-models

In [0]:
from sparknlp.pretrained import PretrainedPipeline

In [0]:
pipeline = PretrainedPipeline('explain_document_ml', lang='en')

NameError: ignored

In [0]:
testDoc = """French author who helped pioner the science-fiction genre. \
Verne wrate about space, air, and underwater travel before navigable aircrast and \
practical submarines were invented, and before any means of space travel had been devised."""

In [0]:
testDoc

'French author who helped pioner the science-fiction genre. Verne wrate about space, air, and underwater travel before navigable aircrast and practical submarines were invented, and before any means of space travel had been devised.'

In [0]:
%%time

result = pipeline.annotate(testDoc)

CPU times: user 34.9 ms, sys: 9.77 ms, total: 44.6 ms
Wall time: 2.26 s


In [0]:
result.keys()

dict_keys(['document', 'spell', 'pos', 'lemmas', 'token', 'stems', 'sentence'])

In [0]:
result['sentence']

['French author who helped pioner the science-fiction genre.',
 'Verne wrate about space, air, and underwater travel before navigable aircrast and practical submarines were invented, and before any means of space travel had been devised.']

In [0]:
result['token']

['French',
 'author',
 'who',
 'helped',
 'pioner',
 'the',
 'science-fiction',
 'genre',
 '.',
 'Verne',
 'wrate',
 'about',
 'space',
 ',',
 'air',
 ',',
 'and',
 'underwater',
 'travel',
 'before',
 'navigable',
 'aircrast',
 'and',
 'practical',
 'submarines',
 'were',
 'invented',
 ',',
 'and',
 'before',
 'any',
 'means',
 'of',
 'space',
 'travel',
 'had',
 'been',
 'devised',
 '.']

In [0]:
list(zip(result['token'], result['pos']))

[('French', 'JJ'),
 ('author', 'NN'),
 ('who', 'WP'),
 ('helped', 'VBD'),
 ('pioner', 'NN'),
 ('the', 'DT'),
 ('science-fiction', 'NN'),
 ('genre', 'NN'),
 ('.', '.'),
 ('Verne', 'NNP'),
 ('wrate', 'VBD'),
 ('about', 'IN'),
 ('space', 'NN'),
 (',', ','),
 ('air', 'NN'),
 (',', ','),
 ('and', 'CC'),
 ('underwater', 'JJ'),
 ('travel', 'NN'),
 ('before', 'IN'),
 ('navigable', 'JJ'),
 ('aircrast', 'NN'),
 ('and', 'CC'),
 ('practical', 'JJ'),
 ('submarines', 'NNS'),
 ('were', 'VBD'),
 ('invented', 'VBN'),
 (',', ','),
 ('and', 'CC'),
 ('before', 'IN'),
 ('any', 'DT'),
 ('means', 'NNS'),
 ('of', 'IN'),
 ('space', 'NN'),
 ('travel', 'NN'),
 ('had', 'VBD'),
 ('been', 'VBN'),
 ('devised', 'VBN'),
 ('.', '.')]

In [0]:
list(zip(result['token'], result['lemmas'], result['stems'], result['spell']))

[('French', 'French', 'french', 'French'),
 ('author', 'author', 'author', 'author'),
 ('who', 'who', 'who', 'who'),
 ('helped', 'help', 'help', 'helped'),
 ('pioner', 'pioneer', 'pioneer', 'pioneer'),
 ('the', 'the', 'the', 'the'),
 ('science-fiction', 'sciencefiction', 'sciencefict', 'sciencefiction'),
 ('genre', 'genre', 'genr', 'genre'),
 ('.', '.', '.', '.'),
 ('Verne', 'Verne', 'vern', 'Verne'),
 ('wrate', 'write', 'wrote', 'wrote'),
 ('about', 'about', 'about', 'about'),
 ('space', 'space', 'space', 'space'),
 (',', ',', ',', ','),
 ('air', 'air', 'air', 'air'),
 (',', ',', ',', ','),
 ('and', 'and', 'and', 'and'),
 ('underwater', 'underwater', 'underwat', 'underwater'),
 ('travel', 'travel', 'travel', 'travel'),
 ('before', 'before', 'befor', 'before'),
 ('navigable', 'navigable', 'navig', 'navigable'),
 ('aircrast', 'aircraft', 'aircraft', 'aircraft'),
 ('and', 'and', 'and', 'and'),
 ('practical', 'practical', 'practic', 'practical'),
 ('submarines', 'submarine', 'submarin', '

In [0]:
df = pd.DataFrame({"token":result['token'],
                   "corrected":result['spell'],
                   'POS':result['pos'],
                   'lemmas':result['lemmas'],
                   'stems':result['stems']}) 
df

,token,corrected,POS,lemmas,stems
0,French,French,JJ,French,french
1,author,author,NN,author,author
2,who,who,WP,who,who
3,helped,helped,VBD,help,help
4,pioner,pioneer,NN,pioneer,pioneer
5,the,the,DT,the,the
6,science-fiction,sciencefiction,NN,sciencefiction,sciencefict
7,genre,genre,NN,genre,genr
8,.,.,.,.,.
9,Verne,Verne,NNP,Verne,vern


## Explain Document DL

In [0]:
pipeline_dl = PretrainedPipeline('explain_document_dl', 'en')

explain_document_dl download started this may take some time.
Approx size to download 168.4 MB
[OK!]


In [0]:
%%time
result = pipeline_dl.annotate(testDoc)

CPU times: user 33.1 ms, sys: 12.7 ms, total: 45.8 ms
Wall time: 724 ms


In [0]:
result.keys()

dict_keys(['entities', 'stem', 'checked', 'lemma', 'document', 'pos', 'token', 'ner', 'embeddings', 'sentence'])

In [0]:
result['entities']

['French', 'Verne']

In [0]:
df = pd.DataFrame({"token":result['token'],
                   "ner_label":result['ner'],
                   "spell_corrected":result['checked'],
                   'POS':result['pos'],
                   'lemma':result['lemma'],
                   'stem':result['stem']}) 
df

,token,ner_label,spell_corrected,POS,lemma,stem
0,French,B-MISC,French,JJ,French,french
1,author,O,author,NN,author,author
2,who,O,who,WP,who,who
3,helped,O,helped,VBD,help,help
4,pioner,O,pioneer,NN,pioneer,pioneer
5,the,O,the,DT,the,the
6,science-fiction,O,sciencefiction,NN,sciencefiction,sciencefict
7,genre,O,genre,NN,genre,genr
8,.,O,.,.,.,.
9,Verne,B-PER,Verne,NNP,Verne,vern


## Spell Checking

In [0]:
spell_checker = PretrainedPipeline('check_spelling', 'en')

check_spelling download started this may take some time.
Approx size to download 892.6 KB
[OK!]


In [0]:
result = spell_checker.annotate(testDoc)

In [0]:
result.keys()

dict_keys(['document', 'sentence', 'token', 'checked'])

In [0]:
list(zip(result['token'], result['checked']))

[('French', 'French'),
 ('author', 'author'),
 ('who', 'who'),
 ('helped', 'helped'),
 ('pioner', 'pioneer'),
 ('the', 'the'),
 ('science-fiction', 'science-fiction'),
 ('genre', 'genre'),
 ('.', '.'),
 ('Verne', 'Vern'),
 ('wrate', 'wrote'),
 ('about', 'about'),
 ('space', 'space'),
 (',', ','),
 ('air', 'air'),
 (',', ','),
 ('and', 'and'),
 ('underwater', 'underwater'),
 ('travel', 'travel'),
 ('before', 'before'),
 ('navigable', 'navigable'),
 ('aircrast', 'aircraft'),
 ('and', 'and'),
 ('practical', 'practical'),
 ('submarines', 'submarines'),
 ('were', 'were'),
 ('invented', 'invented'),
 (',', ','),
 ('and', 'and'),
 ('before', 'before'),
 ('any', 'any'),
 ('means', 'means'),
 ('of', 'of'),
 ('space', 'space'),
 ('travel', 'travel'),
 ('had', 'had'),
 ('been', 'been'),
 ('devised', 'devised'),
 ('.', '.')]

## Parsing a list of texts

In [0]:
testDoc_list = [ 'French author who helped pioner the science-fiction genre.',
'Verne wrate about space, air, and underwater travel before navigable aircrast',
'Practical submarines were invented, and before any means of space travel had been devised.']

In [0]:
result_list = pipeline.annotate(testDoc_list)

In [0]:
len(result_list)

3

In [0]:
result_list[0]

{'document': ['French author who helped pioner the science-fiction genre.'],
 'lemmas': ['French',
  'author',
  'who',
  'help',
  'pioneer',
  'the',
  'sciencefiction',
  'genre',
  '.'],
 'pos': ['JJ', 'NN', 'WP', 'VBD', 'NN', 'DT', 'NN', 'NN', '.'],
 'sentence': ['French author who helped pioner the science-fiction genre.'],
 'spell': ['French',
  'author',
  'who',
  'helped',
  'pioneer',
  'the',
  'sciencefiction',
  'genre',
  '.'],
 'stems': ['french',
  'author',
  'who',
  'help',
  'pioneer',
  'the',
  'sciencefict',
  'genr',
  '.'],
 'token': ['French',
  'author',
  'who',
  'helped',
  'pioner',
  'the',
  'science-fiction',
  'genre',
  '.']}

## Using fullAnnotate to get more details

In [0]:
text = 'Peter Parker is a nice guy and lives in New York'

In [0]:
detailed_result = pipeline_dl.fullAnnotate(text)

In [0]:
detailed_result

[{'checked': [Annotation(token, 0, 4, Peter, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 6, 11, Parker, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 13, 14, is, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 16, 16, a, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 18, 21, nice, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 23, 25, guy, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 27, 29, and, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 31, 35, lives, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 37, 38, in, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 40, 42, New, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 44, 47, York, {'confidence': '1.0', 'sentence': '0'})],
  'document': [Annotation(document, 0, 47, Peter Parker is a nice guy and lives in New York, {})],
  'embeddings': [Annotation(word_embeddings, 0, 4, Peter, {'is

In [0]:
detailed_result[0]['entities']

[Annotation(chunk, 0, 11, Peter Parker, {'entity': 'PER', 'sentence': '0', 'chunk': '0'}),
 Annotation(chunk, 40, 47, New York, {'entity': 'LOC', 'sentence': '0', 'chunk': '1'})]

In [0]:
detailed_result[0]['entities'][0]

Annotation(chunk, 0, 11, Peter Parker, {'entity': 'PER', 'sentence': '0', 'chunk': '0'})

In [0]:
# To reach the value we use .result method
detailed_result[0]['entities'][0].result

'Peter Parker'

In [0]:
detailed_result[0]['entities'][0].metadata['entity']

'PER'

In [0]:
chunks = []
entities = []

for n in detailed_result[0]['entities']:
    chunks.append(n.result)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities})

In [0]:
df

,sent_id,token,start,end,pos,ner
0,0,Peter,0,4,NNP,B-PER
1,0,Parker,6,11,NNP,I-PER
2,0,is,13,14,VBZ,O
3,0,a,16,16,DT,O
4,0,nice,18,21,JJ,O
5,0,guy,23,25,NN,O
6,0,and,27,29,CC,O
7,0,lives,31,35,NNS,O
8,0,in,37,38,IN,O
9,0,New,40,42,NNP,B-LOC


In [0]:
tuples = []

for x, y, z in zip(detailed_result[0]["token"], detailed_result[0]['pos'], detailed_result[0]['ner']):
    tuples.append((int(x.metadata['sentence']), x.result, x.begin, x.end, y.result, z.result))

df = pd.DataFrame(tuples, columns=['sent_id', 'token', 'start', 'end', 'pos', 'ner'])
df

,sent_id,token,start,end,pos,ner
0,0,Peter,0,4,NNP,B-PER
1,0,Parker,6,11,NNP,I-PER
2,0,is,13,14,VBZ,O
3,0,a,16,16,DT,O
4,0,nice,18,21,JJ,O
5,0,guy,23,25,NN,O
6,0,and,27,29,CC,O
7,0,lives,31,35,NNS,O
8,0,in,37,38,IN,O
9,0,New,40,42,NNP,B-LOC


## Use Pretrained **match_chunk** Pipeline for individual Noun Phrase

pipeline uses the regex pattern of `<DT>?<JJ>*<NN>+` (yani determinat (DT) etiher the or a might occur, one or more adjective (JJ) and noun (NN) 

In [0]:
pipeline = PretrainedPipeline('match_chunks', 'en')


match_chunks download started this may take some time.
Approx size to download 4.3 MB
[OK!]


In [0]:
result = pipeline.annotate('The book has many chapters. The red car seems awesome')

In [0]:
result

{'chunk': ['The book', 'The red car'],
 'document': ['The book has many chapters. The red car seems awesome'],
 'pos': ['DT', 'NN', 'VBZ', 'JJ', 'NNS', '.', 'DT', 'JJ', 'NN', 'VBZ', 'JJ'],
 'sentence': ['The book has many chapters.', 'The red car seems awesome'],
 'token': ['The',
  'book',
  'has',
  'many',
  'chapters',
  '.',
  'The',
  'red',
  'car',
  'seems',
  'awesome']}

In [0]:
result = pipeline.annotate('the little brown pitty bird was trying to fly and the green dog was barking at it')

In [0]:
result

{'chunk': ['the little brown pitty bird', 'the green dog'],
 'document': ['the little brown pitty bird was trying to fly and the green dog was barking at it'],
 'pos': ['DT',
  'JJ',
  'JJ',
  'NN',
  'NN',
  'VBD',
  'VBG',
  'TO',
  'VB',
  'CC',
  'DT',
  'JJ',
  'NN',
  'VBD',
  'VBG',
  'IN',
  'PRP'],
 'sentence': ['the little brown pitty bird was trying to fly and the green dog was barking at it'],
 'token': ['the',
  'little',
  'brown',
  'pitty',
  'bird',
  'was',
  'trying',
  'to',
  'fly',
  'and',
  'the',
  'green',
  'dog',
  'was',
  'barking',
  'at',
  'it']}

In [0]:
result['chunk']

['the little brown pitty bird']

## Extract Exact Dates afrom Referential Date Phrahes

In [0]:
pipeline = PretrainedPipeline('match_datetime', 'en')

match_datetime download started this may take some time.
Approx size to download 12.9 KB
[OK!]


In [0]:
result = pipeline.annotate('I saw him yesterday and he told me that he would visit us next week.')

In [0]:
result

{'date': ['2020/04/27', '2020/04/19'],
 'document': ['I saw him yesterday and he told me that he would visit us next week.'],
 'sentence': ['I saw him yesterday and he told me that he would visit us next week.'],
 'token': ['I',
  'saw',
  'him',
  'yesterday',
  'and',
  'he',
  'told',
  'me',
  'that',
  'he',
  'would',
  'visit',
  'us',
  'next',
  'week',
  '.']}

In [0]:
result = pipeline.fullAnnotate('I saw him yesterday and he told me that he would visit us next week.')

In [0]:
result

[{'date': [Annotation(date, 58, 66, 2020/04/27, {'sentence': '0'}),
   Annotation(date, 10, 18, 2020/04/19, {'sentence': '0'})],
  'document': [Annotation(document, 0, 67, I saw him yesterday and he told me that he would visit us next week., {})],
  'sentence': [Annotation(document, 0, 67, I saw him yesterday and he told me that he would visit us next week., {'sentence': '0'})],
  'token': [Annotation(token, 0, 0, I, {'sentence': '0'}),
   Annotation(token, 2, 4, saw, {'sentence': '0'}),
   Annotation(token, 6, 8, him, {'sentence': '0'}),
   Annotation(token, 10, 18, yesterday, {'sentence': '0'}),
   Annotation(token, 20, 22, and, {'sentence': '0'}),
   Annotation(token, 24, 25, he, {'sentence': '0'}),
   Annotation(token, 27, 30, told, {'sentence': '0'}),
   Annotation(token, 32, 33, me, {'sentence': '0'}),
   Annotation(token, 35, 38, that, {'sentence': '0'}),
   Annotation(token, 40, 41, he, {'sentence': '0'}),
   Annotation(token, 43, 47, would, {'sentence': '0'}),
   Annotation(to

## Sentiment Analysis

In [0]:
pipeline =PretrainedPipeline('analyze_sentiment', 'en')

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]


In [0]:
result = pipeline.annotate('The movie I watched was not a good one')

In [0]:
result

{'checked': ['The', 'movie', 'I', 'watched', 'was', 'not', 'a', 'good', 'one'],
 'document': ['The movie I watched was not a good one'],
 'sentence': ['The movie I watched was not a good one'],
 'sentiment': ['negative'],
 'token': ['The', 'movie', 'I', 'watched', 'was', 'not', 'a', 'good', 'one']}

In [0]:
result['sentiment']

['negative']